This Jupyter Notebook analyzes the photometric data obtained by the GoChile telescope for the asteroid 22 Kalliope. Using the data, it performs Lomb-Scargle analysis to obtain rotational period of the asteroid. Furthermore, it prepares the lightcurve data into the format expected by the codes for convex lightcurve inversion developed by M. Kaasalainen and J. Ďurech (see Kaasalainen & Torppa (2001) and Kaasalainen et al. (2001)). The output file of this codes is translated into an .obj file and the 3D model of the asteroid 22 Kalliope is plotted.

In [ ]:
"""
This code cell reads the input file with the results from the differential aperture photometry done on the
images of the asteroid 22 Kalliope in software AstroImageJ, and extracts the columns relevant for further 
data analysis and saves them in a separate .txt file.
Inputs:
-------
    'file.tbl' : .tbl file produced by AstroImageJ

Outputs:
--------
    'data.txt' : text file with extracted columns relavant for further analysis
"""

import pandas as pd
import numpy as np

file_path= 'file.tbl'
table_data= pd.read_csv(file_path, delimiter='\t')

JD_UTC_column= table_data['JD_UTC'].values
Source_AMag_T1_column= table_data['Source_AMag_T1'].values
Source_AMag_Err_T1_column= table_data['Source_AMag_Err_T1'].values
Source_SNR_T1= table_data['Source_SNR_T1'].values

combined= np.column_stack((JD_UTC_column, Source_AMag_T1_column, Source_AMag_Err_T1_column, Source_SNR_T1))
np.savetxt('data.txt', combined, fmt='%.8f', delimiter=',', header='JD_UTC_column, Source_AMag_T1_column, Source_AMag_Err_T1_column, Source_SNR_T1')

In [ ]:
"""
This code cell reads the data from the differential aperture photometry, sorts it by time
(important for spline method) and then using the spline (due to complex trend), removes the 
long-term trend from the lightcurve, which in this case is the shift in apparent magnitude due
to change of distance and position of the asteroid w.r.t. Earth and Sun. Both original and 
detrended lightcurves are plotted in one graph.
Inputs:
-------
    'data.txt' : text file with all the data needed for the lightcurve

Outputs:
--------
    'lightcurve_org_detrend.png' : graph of the original lightcurve and inset detrended lightcurve
"""

import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import LSQUnivariateSpline

#reading the data
data= np.loadtxt('data.txt', delimiter=',', skiprows=1)
time= data[:, 0]
magnitude= data[:, 1]
errmag= data[:, 2]
snr= data[:, 3]

#sorting the data
sorted_indices= np.argsort(time)
sorted_time= time[sorted_indices]
sortedd_time= sorted_time-int(sorted_time[0])
sorted_magnitude= magnitude[sorted_indices]
sorted_errmag= errmag[sorted_indices]

#detrending the data
num_knots= 18
knots= np.linspace(min(sortedd_time), max(sortedd_time), num_knots)
spline= LSQUnivariateSpline(sortedd_time, sorted_magnitude, knots[1:-1])
fitted_trend= spline(sortedd_time)
detrended_magnitude= sorted_magnitude-fitted_trend

#graphing the light curve and fitted trend
plt.figure(figsize=(16, 6))
plt.errorbar(sortedd_time, sorted_magnitude, yerr=sorted_errmag, fmt='o', color='black', 
            label='Original Data', markersize=1, ecolor='gray', capsize=2)
plt.plot(sortedd_time, fitted_trend, color='gray', alpha=0.5, label='Fitted Trend')
plt.xlabel(f'JD UTC -{int(sorted_time[0])}')
plt.ylabel('Magnitude')
plt.gca().invert_yaxis()
plt.legend(loc='lower right')

#graphing a subgraph with detrended magnitude
inset_axes= plt.axes([0.145, 0.59, 0.37, 0.3])
inset_axes.errorbar(sortedd_time, detrended_magnitude, yerr=sorted_errmag, fmt='o', color='black',
                    label='Detrended Data', markersize=1, ecolor='gray', capsize=2)
inset_axes.set_xlabel(f'JD UTC -{int(sorted_time[0])}')
inset_axes.set_ylabel('Magnitude', labelpad=0.1)
inset_axes.invert_yaxis()
inset_axes.legend()
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)
plt.savefig('lightcurve_org_detrend.png')
plt.show()

In [ ]:
"""
This code cell uses the data from previous code cell to perform Lomb-Scargle analysis 
to determine the rotational period of the asteroid. It produces Lomb-Scargle periodogram
and phased lightcurve with a fit.
Inputs:
-------
    sortedd_time : sorted JD UTC time minus the starting day of the observations
    detrended_magnitude : detrended apparent magnitude of the asteroid sorted by time
    sorted_errmag : sorted error of detrended_magnitude
Outputs:
--------
    'lombscargle.png' : Lomb-Scargle periodogram
    'phase.png' : phase lightcurve and fit to the data
"""

from astropy.timeseries import LombScargle

#Lomb-Scargle
frequency= np.linspace(3, 48, 30000)
nterms= 7
ls= LombScargle(sortedd_time, detrended_magnitude, sorted_errmag, nterms=nterms)
power= ls.power(frequency)

#determining the best frequency and translating that into period
best_frequency= frequency[np.argmax(power)]
best_period_days= 1.0/best_frequency
best_period_hours= best_period_days*24
print(f'Best period: {best_period_hours:.4f} hours')

#plotting the Lomb-Scargle power against PERIOD
period_hours = 24.0/frequency
plt.figure(figsize=(10, 6))
plt.plot(period_hours, power, '-k')
plt.xlim(0.5, 8)
plt.ylim(-0.05, 1.05)
plt.xlabel('Period (hours)')
plt.ylabel('Lomb-Scargle Power')
plt.savefig('lombscargle.png')
plt.show()

#plotting the phased magnitude and phase fit
phase= (sortedd_time*best_frequency)%1
plt.figure(figsize=(10, 6))
plt.errorbar(phase, detrended_magnitude, yerr=sorted_errmag, fmt='ok', ecolor='gray', 
            label='Measured data', capsize=1, alpha=0.2, markersize=2)
phase_fit= np.linspace(0, 1, 200)
y_fit= ls.model(phase_fit/best_frequency, best_frequency)
plt.plot(phase_fit, y_fit, '-', lw=2, color='black', label='Phase fit')
plt.xlim(0, 1)
plt.ylim(np.min(detrended_magnitude)-0.05, np.max(detrended_magnitude)+0.05)
plt.xlabel('Phase')
plt.ylabel('Magnitude')
plt.gca().invert_yaxis()
plt.legend()
plt.savefig('phase.png')
plt.show()

In [ ]:
"""
This code cell generates text file in the format requested by the instructions of the
code for for convex lightcurve inversion method developed by M. Kaasalainen and J. Ďurech
(see Kaasalainen & Torppa (2001) and Kaasalainen et al. (2001)).
Inputs:
-------
    time : JD UTC time of the observation
    magnitude : apparent magnitude of the asteroid
    calib : 0 for relative lightcurve and 1 for calibrated lightcurve
Outputs:
    'data_3dmodel.txt' : text file that contains JD UTC time with light-time correction,
                         relative intensity, and Cartesian coordinates of Sun and Earth
--------
"""

from astroquery.jplhorizons import Horizons
from astropy.time import Time
import astropy.units as u
import numpy as np

data_len= len(time)
jd_utc= Time(time, format='jd', scale='utc')
intensity= 10**(-0.4*magnitude)

calib= 0  # 0 for relative lightcurve and 1 for calibrated

#light-time correction
def light_time_correction(jdutc):
    gochile= {'lon': 289.2369*u.deg, 'lat': -30.4725*u.deg, 'elevation': 1.525*u.km}
    asteroid_obj= Horizons(id='22', location=gochile, epochs=jdutc.jd, id_type='smallbody')
    asteroid_eph= asteroid_obj.vectors()
    light_time= asteroid_eph['lighttime'][0]
    corrected_jd= jdutc.jd - light_time
    return corrected_jd

#acquiring coordinates
sun_coord= []
earth_coord= []
corrected_jd_list= []

for jd in jd_utc:
    corrected_jd= light_time_correction(jd)
    corrected_jd_list.append(corrected_jd)

    #asteroid's position relative to the Sun
    asteroid_obj= Horizons(id='22', location='@sun', epochs=corrected_jd)
    asteroid_eph= asteroid_obj.vectors(refplane='ecliptic')
    asteroid_position= np.array([asteroid_eph['x'][0], asteroid_eph['y'][0], asteroid_eph['z'][0]])
    sun_position= np.array([0, 0, 0])

    #Earth's position relative to the Sun
    earth_obj= Horizons(id='399', location='@sun', epochs=corrected_jd)
    earth_eph= earth_obj.vectors(refplane='ecliptic')
    earth_position= np.array([earth_eph['x'][0], earth_eph['y'][0], earth_eph['z'][0]])

    #relative positions with respect to the asteroid
    sun_relative= sun_position-asteroid_position
    earth_relative= earth_position-asteroid_position
    sun_coord.append(sun_relative)
    earth_coord.append(earth_relative)

#writing the required text file
file= 'data_3dmodel.txt'
with open(file, 'w') as f:
    f.write(f'{data_len} {calib}\n')
    for i in range(data_len):
        f.write(
            f'{corrected_jd_list[i]:.15f} {intensity[i]:.15f}\t'
            f'{sun_coord[i][0]:.8f} {sun_coord[i][1]:.8f} {sun_coord[i][2]:.8f}\t'
            f'{earth_coord[i][0]:.8f} {earth_coord[i][1]:.8f} {earth_coord[i][2]:.8f}\n')

In [ ]:
"""
This code cell reads the output file by the convex lightcurve inversion method codes
developed by M. Kaasalainen and J. Ďurech (see Kaasalainen & Torppa (2001) and 
Kaasalainen et al. (2001)), parses it, produces .obj file to be able to display 3D model,
and plots the 3D model of the asteroid, and its projections on xy, yz and xz axis.
Inputs:
-------
    'model' : output file of the standardtri code developed by M. Kaasalainen and J. Ďurech
Outputs:
--------
    'model.obj' : .obj file with the 3D model of the asteroid
    '3dmodel.png' : 3D graph of the model of the asteroid
    'projections_views.png' : X, Y, and Z projections of the model
"""

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

#reading vertices and faces from the input file
def parse_model(file_path):
    vertices= []
    faces= []

    with open(file_path, 'r') as file:
        first_line= file.readline().strip().split()
        num_vertices= int(first_line[0])
        num_faces= int(first_line[1])

        for _ in range(num_vertices):
            line= file.readline().strip().split()
            vertices.append(list(map(float, line)))

        for _ in range(num_faces):
            line= file.readline().strip().split()
            face= [int(vertex)-1 for vertex in line[1:]]
            faces.append(face)

    return np.array(vertices), faces

#using vertices and faces to produce output .obj file
def convert_to_obj(vertices, faces, output_file_path):
    with open(output_file_path, 'w') as f:
        for vertex in vertices:
            x, y, z = vertex
            f.write(f'v {x} {y} {z}\n')
        
        for face in faces:
            face_indices= [str(index+1) for index in face]
            f.write(f"f {' '.join(face_indices)}\n")

#plotting a 3D graph with the model of the asteroid
def plot_3d_model(vertices, faces, save_path='3dmodel.png'):
    fig= plt.figure()
    ax= fig.add_subplot(111, projection='3d')
    
    mesh= Poly3DCollection([vertices[face] for face in faces], alpha=0.5, edgecolor='k')
    ax.add_collection3d(mesh)
    
    scale= vertices.flatten()
    ax.auto_scale_xyz(scale, scale, scale)

    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])
    
    plt.savefig(save_path, dpi=200)
    plt.show()

#plotting xy, xz and yz projections of the model of the asteroid
def plot_3d_model_from_views(vertices, faces, view_angles, save_path='projections_views.png'):
    fig= plt.figure(figsize=(15, 5))

    for i, view_angle in enumerate(view_angles):
        ax= fig.add_subplot(1, 3, i+1, projection='3d')

        mesh= Poly3DCollection([vertices[face] for face in faces], alpha=0.5, edgecolor='k')
        ax.add_collection3d(mesh)

        scale= vertices.flatten()
        ax.auto_scale_xyz(scale, scale, scale)

        ax.view_init(elev=view_angle[0], azim=view_angle[1])

        ax.set_title(f'View from {["X", "Y", "Z"][i]}-axis')
       
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_zticks([])

    plt.tight_layout()
    plt.savefig(save_path, dpi=200)
    plt.show()

input_file= 'model' 
obj_file= 'model.obj'  

vertices, faces= parse_model(input_file)
convert_to_obj(vertices, faces, obj_file)

view_angles= [(0, 90), (0, 0), (90, 0)]
plot_3d_model(vertices, faces)
plot_3d_model_from_views(vertices, faces, view_angles)